In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('imdb.csv',names=['Text','Label'],header=None)
df.head()

,Text,Label
0,"I didn't expect much from this film, but oh br...",2
1,Unashamedly ambitious sci-fi from Kerry Conran...,2
2,This adaptation positively butchers a classic ...,2
3,"I really did like this show, once upon a time....",2
4,"This movie was horrible, simply put. It was so...",2


In [8]:
df1 = df[df.Label != 3]
print(len(df1))

25000


In [10]:
df_test = pd.read_csv('testing_set.csv',names=['Text','Label'],header=None)
df_test1 = df_test[df_test.Label != 3]
print(len(df_test1))

25000


In [11]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB
from sklearn.metrics import confusion_matrix

import numpy as np
import itertools
import matplotlib.pyplot as plt

In [12]:
def preprocessing_data(data):
    for i in range(len(data)):
        data['Text'][i] = data['Text'][i].lower()
        data['Text'][i] = data['Text'][i].replace('\n',' ')
        data['Text'][i] = ''.join(c for c in data['Text'][i] if c.isalpha() | (c == ' '))
        #print(data['Lyrics'][i])
    return data 

In [13]:
df1 = preprocessing_data(df1)
df1.head()

<ipython-input-12-8b0f78658c8d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Text'][i] = data['Text'][i].lower()
<ipython-input-13-ec49294b0bc3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1 = preprocessing_data(df1)
<ipython-input-12-8b0f78658c8d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Text'][i] = data['Text'][i].replace('\n',' ')
<ipython-input-12-8b0f78658c8d>:5: SettingWithCopyWarning: 
A value is t

,Text,Label
0,i didnt expect much from this film but oh brot...,2
1,unashamedly ambitious scifi from kerry conran ...,2
2,this adaptation positively butchers a classic ...,2
3,i really did like this show once upon a time t...,2
4,this movie was horrible simply put it was so b...,2


In [14]:
df_test1 = preprocessing_data(df_test1)
df_test1.head()

<ipython-input-12-8b0f78658c8d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Text'][i] = data['Text'][i].lower()
<ipython-input-12-8b0f78658c8d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Text'][i] = data['Text'][i].replace('\n',' ')
<ipython-input-12-8b0f78658c8d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Text'][i] = ''.join(c for c in data['Text'][i] if c.isalpha() | (c == ' '))


,Text,Label
0,classes tous risques is one of the best gangst...,1
1,spunky journalist holly hunter produces invest...,1
2,i tend to get furious when hearing about lucio...,1
3,the planet is an astounding piece of film maki...,1
4,i recall so many things about seeing this movi...,1


In [26]:
def split_data(data, training_ratio):
    total = len(data)
    training_data = []
    evaluation_data = []

    for indice in range(0, total):
        if indice < total * training_ratio:
            training_data.append([data['Text'][indice],data['Label'][indice]])
        else:
            evaluation_data.append([data['Text'][indice],data['Label'][indice]])

    return training_data, evaluation_data

In [28]:
# train_data, eval_data = split_data(df)
train_data, no_matter = split_data(df1,1)
eval_data, no_matter2 = split_data(df_test1,1)

In [30]:
print(len(train_data))
print(len(eval_data))

25000
25000


In [31]:
def training_step(training_data, vectorizer):
    training_text = []
    training_result = []
    for i in range(len(training_data)):
        training_text.append(training_data[i][0])
        training_result.append(training_data[i][1])
    training_text = vectorizer.fit_transform(training_text) 
    
#     print("Training text: " + str(training_text))
#     print("Training result: " + str(training_result))
#     for result in training_result: 
#         if result == "nan": 
#             print("Error there is a nan")
#             result = 0
#         else: 
#             print("Fine")
            
            
    classifier = MultinomialNB().fit(training_text, training_result) #remove toarray for bernoulli/multi
#     print("Classifier: " + str(classifier))
#     print("Classifier log prior: " + str(classifier.class_log_prior_))
#     print("Feature log prob: " + str(classifier.feature_log_prob_))
    classifier.get_params()
    return classifier

In [32]:
def analyze_text(classifier, vectorizer, text):
#     print("Classifier prediction: " + str(classifier.predict(vectorizer.transform([text]))))
    return text, classifier.predict(vectorizer.transform([text]))

In [33]:
def evaluate(evaluation_data):
    evaluation_text = []
    actual_result = []
    for i in range(len(evaluation_data)):
        evaluation_text.append(evaluation_data[i][0])
        actual_result.append(int(evaluation_data[i][1]))

    total = len(evaluation_text)
    corrects = 0
    for i in range(0, total):
        result = classifier.predict(vectorizer.transform([evaluation_text[i]])) #remove toarray
        text = evaluation_text[i]
        corrects += 1 if result[0] == actual_result[i] else 0
    print("Accuracy: ", corrects * 100 / total)
    return "Accuracy: ", corrects * 100 / total

In [34]:
v = [CountVectorizer(binary = 'true'), CountVectorizer(), TfidfVectorizer(binary = 'true'), TfidfVectorizer()]
for vectorizer in v: 
    classifier = training_step(train_data, vectorizer)
    evaluate(eval_data)

Accuracy:  82.732
Accuracy:  81.52
Accuracy:  83.908
Accuracy:  83.116


In [35]:
def create_confusion_matrix(evaluation_data):
    evaluation_text     = [evaluation_data[0] for evaluation_data in evaluation_data]
    actual_result       = [int(evaluation_data[1]) for evaluation_data in evaluation_data]
    prediction_result   = []
#     print(actual_result)
    for text in evaluation_text:
        analysis_result = analyze_text(classifier, vectorizer, text)
        prediction_result.append(int(analysis_result[1][0]))
#     print(prediction_result)
#     print(evaluation_text[9])
    matrix = confusion_matrix(actual_result, prediction_result)
    return matrix

In [36]:
result = create_confusion_matrix(eval_data)
pd.DataFrame(
    result, 
    columns=["Negatives", "Positives"],
    index=["Negatives", "Positives"])

,Negatives,Positives
Negatives,9636,2864
Positives,1357,11143


In [ ]:
def print_result(result):
    text, analysis_result = result
    print(analysis_result)
    print_text = "Positive" if analysis_result[0] == 1 else "Negative"
    print(text, ":", print_text)

In [ ]:
print_result(analyze_text(classifier, vectorizer, "i cant wait"))